# Chest X-ray Classification (Pneumonia/COVID-19)
This notebook demonstrates data loading, training, evaluation, and inference with Grad-CAM.

Requirements:
- Dataset in `data/` with subfolders per class (e.g., Normal, Pneumonia, COVID-19)
- GPU recommended; Apple Silicon supported via `tensorflow-metal`


In [ ]:
%pip install -q -r ../../requirements.txt


In [ ]:
import os
import tensorflow as tf
from src.medimg.data import create_datasets
from src.medimg.models import build_scratch_cnn, build_resnet50, build_efficientnetb0
from src.medimg.utils import compile_model, evaluate_model, compute_gradcam_heatmap


In [ ]:
data_dir = '/path/to/data'  # TODO: set
train_ds, val_ds, test_ds, class_names = create_datasets(data_dir)
input_shape = (224, 224, 3)
num_classes = len(class_names)
model = compile_model(build_efficientnetb0(input_shape, num_classes), num_classes)
history = model.fit(train_ds, validation_data=val_ds, epochs=3)
res = evaluate_model(model, test_ds, class_names)
print('Accuracy:', (res['y_true'] == res['y_pred']).mean())
print('ROC-AUC:', res['roc_auc'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.medimg.utils import plot_confusion_matrix, plot_roc_curve, compute_gradcam_heatmap

cm_fig = plot_confusion_matrix(res['confusion_matrix'], class_names)
plt.show(cm_fig)
if res['fpr'] is not None:
    roc_fig = plot_roc_curve(res['fpr'], res['tpr'], res['roc_auc'])
    plt.show(roc_fig)

# Grad-CAM on a single sample
for batch_x, batch_y in test_ds.take(1):
    x0 = batch_x[0:1]
    heatmap = compute_gradcam_heatmap(model, x0)
    print('Pred:', np.argmax(model.predict(x0, verbose=0)))
    break
